# **`IMPLEMENTATION FROM SCRATCH DES K VOISINS LES PLUS PROCHES`**

L'algorithme **KNN (K-nearest neighbors)** est un modèle de Machine Learning supervisé, c'est-à-dire qui se base sur des données d'entrée étiquetées.<br> 
C'est en apprenant de ce type de données qu'il sera en mesure de proposer une sortie appropriée en partant maintenant de nouvelles données non étiquetées. L'algorithme peut être appliqué avec le langage de programmation Python à partir de fichier csv. KNN des plus proches voisins s'utilise pour la **régression** et la **classification** des données.

Dans la pratique, KNN n'aura pas besoin de passer par la construction d'un modèle prédictif pour réaliser des prédictions. L'algorithme classe les nouvelles données en se basant sur le jeu de données précédant pour fournir des résultats. Son principe se décrit par cette affirmation : **« Dis-moi qui sont tes voisins, je te dirais qui tu es »**.



## **`RESUME DES DIFFERENTES ETAPES DE LA METHODOLOGIE CRISP-DM`**
### **`I. DESCRIPTION ET OBJECTIF DE NOTRE ETUDE`**
### **`II. IMPLEMENTATION ET APPLICATION DES KNN POUR LA CLASSIFICATION`**
#### **`II.1. IMPLEMENTATION DES KNN POUR LA CLASSIFICATION`**  
#### **`II.2. APPLICATION DES KNN POUR LA CLASSIFICATION DANS LA BASE DE DONNEES IRIS`**

### **`III. IMPLEMENTATION DES KNN POUR LA REGRESSION`**


# **`I. DESCRIPTION ET OBJECTIF DE NOTRE ETUDE`**


**-** K-plus proches voisins (KNN) est un type d'algorithme d'apprentissage supervisé utilisé à la fois pour la régression et la classification.<br> 
KNN essaie de prédire la classe correcte pour les données de test en calculant la distance entre les données de test et tous les points d'apprentissage<br> Sélectionnez ensuite le nombre K de points qui est le plus proche des données de test.<br> 

L'algorithme KNN calcule la probabilité que les données de test appartiennent aux classes de données d'apprentissage 'K' et que la classe détient la probabilité la plus élevée sera sélectionnée. Dans le cas d'une régression, la valeur est la moyenne des 'K' points d'apprentissage sélectionnés.<br>

L’algorithme k-NN est une méthode non-paramétrique d’apprentissage supervisé qui peut être utilisée pour résoudre des problèmes de régression et de classification. Il est basé sur la similarité entre les instances de données et est intuitif et facile à comprendre.



**-** C'est dans cette perspective que nous nous fixons comme objectif d'implémenter from scratch l’algorithme des k voisins les plus proches (k-NN) pour la régression et la classification en utilisant la programmation orientée objet et en tenant compte des hyperparamètres tels que la distance (**euclidienne ou manhattan**),le nombre de voisins(k), la pondération des votes(**distance ou uniforme**), la normalisation des caractéristiques et la méthode d'indexation spatiale(**k-d tree ou ball tree**) .<br>


# **`II. IMPLEMENTATION ET APPLICATION DES (KNN) POUR LA CLASSIFICATION`**


## **`II.1. IMPLEMENTATION DES KNN POUR LA CLASSIFICATION`**


In [1]:
# Importation des librairies utiles
import numpy as np

from collections import Counter                     # Pou le compte des k


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler   # Pour la normalisation des caractéristiques
from sklearn.neighbors import KDTree, BallTree     # Pour la méthode d'indexation

import warnings                                    # Pour éviter la génération des warnings 
warnings.filterwarnings('ignore')

In [2]:
class KnnClassification:
    # Constructeur
    
    def __init__(self, distance ='euclidienne',k=3, ponderation='distance', methode_indexation='kd_tree'):

        self.k = k
        self.distance = distance
        self.ponderation = ponderation
        self.methode_indexation = methode_indexation
        self.X_train = None
        self.y_train = None
        self.tree = None

    # Fonction d'entrainement

    def entrainement(self, X_train, y_train):
    
        self.X_train = X_train
        self.y_train = y_train

        if self.methode_indexation == 'kd_tree':
            self.tree = KDTree(self.X_train)
        elif self.methode_indexation == 'ball_tree':
            self.tree = BallTree(self.X_train)
        else:
            raise ValueError("Vos méthodes d'indexation sont invalides, il ne faut choisir qu'une parmi : KDTree et BallTree")
        

    # Fonction de calcul de la distance

    def mesure_distance(self, Xi, Yi):
      
        # Distance euclidienne : plus la distance est petite, plus les deux observations sont identiques.
        if ( self.distance == 'euclidienne'):
            distanc = np.sqrt(np.sum((Xi - Yi) ** 2))
            
            return distanc
            
        # Distance manhattan : connue sous le nom de distance en paté de maisons, elle est la somme des differences absolues de leurs coordonnées cartesiennes.
        elif (self.distance == 'manhattan'):
            distanc = np.sum(np.abs(Xi - Yi))

            return distanc
        

    # Fonction de prédiction

    def predictClassif(self, X_test):
        y_pred = []

        for point_test in X_test:
            distances = []

            # Calcul des distances entre l'échantillon de test et tous les échantillons d'entraînement
            for point_train in self.X_train:
                d = self.mesure_distance(point_test, point_train)
                distances.append(d)

            # Tri des distances et récupération des indices des k plus proches voisins
            indices = np.argsort(distances)[:self.k]
            k_indices_proches = np.array(distances)[indices]

            # Calcul des poids inverses des distances
            poids = 1.0 / k_indices_proches

            # Calcul des poids normalisés
            poids = poids / np.sum(poids)

            # Prédiction de l'étiquette en utilisant les poids pondérés
            k_proches = self.y_train[indices]
            k_voisins_ponderes = np.dot(poids, k_proches).flatten()   # Convertir les k_voisins_ponderes en un tableau
            k_voisins_ponderes = np.round(k_voisins_ponderes).astype(int)

            # Supprimer les valeurs négatives de k_voisins_ponderes
            k_voisins_ponderes = [x for x in k_voisins_ponderes if x >= 0]

            # Calculer le mode en utilisant np.bincount()
            if len(k_voisins_ponderes) > 0:
                mode_value = np.argmax(np.bincount(k_voisins_ponderes))
           
            y_pred.append(mode_value)

            predictions = np.array(y_pred)
           
        return predictions
    

    # Fonction de calcul de l'accuracy 

    def accuracy_knn(self, y_vraie, y_pred): 
        correctes = 0
        for i in range(len(y_vraie)): 
            if y_vraie[i] == y_pred[i]: 
                correctes = correctes + 1
        return (correctes / float(len(y_vraie)))*100
 

## **`II.1. APPLICATION DES KNN POUR LA CLASSIFICATION DANS LA BASE DE DONNEES IRIS`**


In [1]:
# Importation du jeu de données
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [2]:
# Récupération du dataset, et partitionnement de ce dernier
iris = datasets.load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
print("Dimensions X_train :", X_train.shape)
print("Dimensions y_train :", y_train.shape)
print("Dimensions X_test :", X_test.shape)
print("Dimensions y_test :", y_test.shape)

Dimensions X_train : (120, 4)
Dimensions y_train : (120,)
Dimensions X_test : (30, 4)
Dimensions y_test : (30,)


In [6]:
# Normalisation du dataset (ou les caratéristiques de la base)
scaler = StandardScaler().fit(X_train) 
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
# Instanciation de notre algorithme
knn = KnnClassification(k=3,distance='euclidienne', ponderation='distance', methode_indexation='kd_tree')
# Entrainement du modèle
knn.entrainement(X_train, y_train)

In [8]:
print(knn.X_train.shape)
print(knn.y_train.shape)

(120, 4)
(120,)


In [9]:
# Prédiction de notre algoriyhme de classification
predictions = knn.predictClassif(X_test)
predictions

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0], dtype=int64)

In [10]:
# Performance de notre modèle 
knn.accuracy_knn(y_test,predictions)

100.0

# **`II. IMPLEMENTATION DES KNN POUR LA REGRESSION`**


Dans cette partie, nous ne calculons plus le mode pour les étiquettes des voisins pondérés. Au lieu de cela, nous utilisons la moyenne pondérée des valeurs cibles des k plus proches voisins pour prédire la valeur continue pour chaque point de test.

In [26]:
class KnnRegression:
    # Constructeur
    
    def __init__(self, distance ='euclidienne',k=3, ponderation='distance', methode_indexation='kd_tree'):

        self.k = k
        self.distance = distance
        self.ponderation = ponderation
        self.methode_indexation = methode_indexation
        self.X_train = None
        self.y_train = None
        self.tree = None

    # Fonction d'entrainement

    def entrainement(self, X_train, y_train):
    
        self.X_train = X_train
        self.y_train = y_train

        if self.methode_indexation == 'kd_tree':
            self.tree = KDTree(self.X_train)
        elif self.methode_indexation == 'ball_tree':
            self.tree = BallTree(self.X_train)
        else:
            raise ValueError("Vos méthodes d'indexation sont invalides, il ne faut choisir qu'une parmi : KDTree et BallTree")
        

    # Fonction de calcul de la distance

    def mesure_distance(self, Xi, Yi):
      
        # Distance euclidienne : plus la distance est petite, plus les deux observations sont identiques.
        if ( self.distance == 'euclidienne'):
            distanc = np.sqrt(np.sum((Xi - Yi) ** 2))
            
            return distanc
            
                    # Distance manhattan : connue sous le nom de distance en paté de maisons, elle est la somme des differences absolues de leurs coordonnées cartesiennes.
        elif (self.distance == 'manhattan'):
            distanc = np.sum(np.abs(Xi - Yi))

            return distanc
    
    # Fonction de prédiction

    def predictReg(self, X_test):
        y_pred = []

        for point_test in X_test:
            distances = []

        # Calcul des distances entre l'échantillon de test et tous les échantillons d'entraînement
            for point_train in self.X_train:
                d = self.mesure_distance(point_test, point_train)
                distances.append(d)

            # Tri des distances et récupération des indices des k plus proches voisins
            indices = np.argsort(distances)[:self.k]
            k_indices_proches = np.array(distances)[indices]

            # Calcul des poids inverses des distances
            poids = 1.0 / k_indices_proches

            # Calcul des poids normalisés
            poids = poids / np.sum(poids)

            # Prédiction de l'étiquette en utilisant les poids pondérés
            k_proches = self.y_train[indices]
            k_voisins_ponderes = np.dot(poids, k_proches).flatten()   # Convertir les k_voisins_ponderes en un tableau
            k_voisins_ponderes = np.round(k_voisins_ponderes).astype(int)
            k_voisins_ponderes = [x for x in k_voisins_ponderes if x >= 0]
            
            # Calculer la moyenne en utilisant np.mean()
            if len(k_voisins_ponderes) > 0:
                prediction = np.mean(k_voisins_ponderes)
            else:
                prediction = 0.0

            y_pred.append(prediction)

        return y_pred

    

    # Fonction de calcul du score

    def score(self , y , y_pred):
        RMSE = np.sqrt(np.mean((y - y_pred)**2))

        return RMSE
    